In [217]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline

In [218]:
df_dev = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_gender = pd.read_csv('gender_submission.csv')


In [219]:
df_dev.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [220]:
df_train = df_dev[['Pclass','Sex','Age','SibSp','Parch','Fare','Cabin','Embarked']]

In [221]:
df_train['Age'] = df_train['Age'].fillna(df_train['Age'].mean())
df_train['Cabin'] = df_train['Cabin'].fillna(-1)
df_train['Embarked'] = df_train['Embarked'].fillna(-1)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pa

In [222]:
for field in ['Sex','Embarked']:
    df_dummy = pd.get_dummies(df_train[field],prefix = field)
    df_train = pd.concat([df_train,df_dummy],axis  = 1)
    df_train = df_train.drop(field,axis = 1)

In [223]:
df_nan = df_train.isnull()
df_nan.describe()

,Pclass,Age,SibSp,Parch,Fare,Cabin,Sex_female,Sex_male,Embarked_-1,Embarked_C,Embarked_Q,Embarked_S
count,891,891,891,891,891,891,891,891,891,891,891,891
unique,1,1,1,1,1,1,1,1,1,1,1,1
top,False,False,False,False,False,False,False,False,False,False,False,False
freq,891,891,891,891,891,891,891,891,891,891,891,891


In [224]:
df_train = df_train.drop('Cabin', axis = 1)

In [225]:
df_train

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_-1,Embarked_C,Embarked_Q,Embarked_S
0,3,22.000000,1,0,7.2500,0,1,0,0,0,1
1,1,38.000000,1,0,71.2833,1,0,0,1,0,0
2,3,26.000000,0,0,7.9250,1,0,0,0,0,1
3,1,35.000000,1,0,53.1000,1,0,0,0,0,1
4,3,35.000000,0,0,8.0500,0,1,0,0,0,1
5,3,29.699118,0,0,8.4583,0,1,0,0,1,0
6,1,54.000000,0,0,51.8625,0,1,0,0,0,1
7,3,2.000000,3,1,21.0750,0,1,0,0,0,1
8,3,27.000000,0,2,11.1333,1,0,0,0,0,1
9,2,14.000000,1,0,30.0708,1,0,0,1,0,0


In [226]:
df_target = df_dev[['Survived']]

In [227]:
df_target

,Survived
0,0
1,1
2,1
3,1
4,0
5,0
6,0
7,0
8,1
9,1


In [228]:
x_np = df_train.as_matrix().astype(np.float32)

In [249]:
df_train.describe()

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_-1,Embarked_C,Embarked_Q,Embarked_S
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,2.308642,29.699118,0.523008,0.381594,32.204208,0.352413,0.647587,0.002245,0.188552,0.086420,0.722783
std,0.836071,13.002015,1.102743,0.806057,49.693429,0.477990,0.477990,0.047351,0.391372,0.281141,0.447876
min,1.000000,0.420000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,22.000000,0.000000,0.000000,7.910400,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.000000,29.699118,0.000000,0.000000,14.454200,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
75%,3.000000,35.000000,1.000000,0.000000,31.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000
max,3.000000,80.000000,8.000000,6.000000,512.329200,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [229]:
y_np = df_target.as_matrix().astype(np.int32)

In [230]:
from sklearn.model_selection import train_test_split
import chainer
from chainer import training, Chain
from chainer.training import extensions
from chainer import links as L
from chainer import functions as F
from chainer.datasets import TupleDataset
from chainer import serializers

In [192]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [396]:
X_train,X_test,y_train,y_test = train_test_split(x_np,y_np,test_size=0.2)

In [397]:
train = TupleDataset(X_train,y_train.reshape(1,-1)[0])
test = TupleDataset(X_test,y_test.reshape(1,-1)[0])

In [398]:
train_iter = chainer.iterators.SerialIterator(train, 100)
test_iter = chainer.iterators.SerialIterator(test, 100,repeat=False, shuffle=False)

In [399]:
class MyChain5(Chain):
    def __init__(self):
        super(MyChain5,self).__init__(
            l1 = L.Linear(None, 100),
            l2 = L.Linear(None, 100),
            l3 = L.Linear(None,100),
            l4 = L.Linear(None,2),
            bn1 = L.BatchNormalization(size = 100),
            bn2 = L.BatchNormalization(size = 100),
            bn3 = L.BatchNormalization(size = 100)
        )
        
    def __call__(self, x):
        h = F.dropout(x,ratio = 0.2)
        h = F.dropout(F.relu(self.bn1(self.l1(x))))
        h = F.dropout(F.relu(self.bn2(self.l2(h))))
        h = F.dropout(F.relu(self.bn3(self.l3(h))))
        y = self.l4(h)
        
        return y


In [400]:
model = L.Classifier(MyChain4())
optimizer = chainer.optimizers.Adam()
optimizer.setup(model)

In [406]:
updater = training.StandardUpdater(train_iter, optimizer, device=-1)
trainer = training.Trainer(updater, (10000, 'epoch'), out="result")
trainer.extend(extensions.Evaluator(test_iter, model, device=-1))
trainer.extend(extensions.LogReport())
trainer.extend(extensions.PrintReport( ['epoch', 'main/loss', 'validation/main/loss', 'main/accuracy', 'validation/main/accuracy']))
#trainer.extend(extensions.ProgressBar())

In [407]:
trainer.run()

epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy
1000        0.418415    0.439095              0.83           0.841392                  
1001        0.370976    0.502114              0.841429       0.815063                  
1002        0.367055    0.447614              0.852857       0.847722                  
1003        0.367912    0.430777              0.848571       0.832405                  
1004        0.400877    0.386767              0.818571       0.840063                  
1005        0.371425    0.437906              0.847143       0.820063                  
1006        0.383524    0.429364              0.84           0.825063                  
1007        0.368856    0.417541              0.842857       0.836392                  
1008        0.38385     0.42576               0.848571       0.840063                  
1009        0.370573    0.445493              0.83875        0.823734                  
1010        0.381091    0.405015  

KeyboardInterrupt: 

In [410]:
serializers.save_npz('model/model_5.npz',model)

In [408]:
pred = np.argmax(F.softmax(model.predictor(X_test)).data,axis = 1)

In [409]:
print classification_report(y_test,pred)

             precision    recall  f1-score   support

          0       0.87      0.86      0.86       118
          1       0.73      0.75      0.74        61

avg / total       0.82      0.82      0.82       179



In [193]:
classifier = SVC()

In [194]:
classifier.fit(X_train,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [195]:
pd = classifier.predict(X_test)

In [196]:
confusion_matrix(y_test,pd)

array([[87, 19],
       [33, 40]])

In [197]:
print classification_report(y_test,pd)

             precision    recall  f1-score   support

          0       0.72      0.82      0.77       106
          1       0.68      0.55      0.61        73

avg / total       0.71      0.71      0.70       179



In [198]:
print accuracy_score(y_test,pd)

0.709497206704
